In [1]:
from lib.config import Config
from lib.data_set import Dataset
from lib.model import NNModel
from lib import utils

In [2]:
import imp
imp.reload(utils)

<module 'lib.utils' from 'E:\\Document\\jupyter\\Master Thesis\\lib\\utils.py'>

In [3]:
import ctypes
import pandas as pd
import numpy as np
import pywt
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers as KL
from tensorflow.keras import models as KM
from tensorflow.keras import backend as K
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow import keras
import tensorflow as tf

## File name read

In [8]:
# read file name of data with various Labels
df = pd.read_csv('./useful_data_label.csv',index_col=0) 
# read file name of data with only label 0
df2 = pd.read_csv('./unuseful_data_label.csv',index_col=0)
# read some of the data with only label 0
df3 = pd.read_csv('./data/file_name.txt',header=None)
# player = ctypes.windll.kernel32

ind = df2.iloc[1].isna()
files = np.concatenate([np.array(df.columns),np.array('normal/'+df2.columns[ind])])

## Configuraion

In [9]:
# Override the base class of Config and Features for CWT-CNN Model
class CNNPa_Config(Config):
    NAME = 'CNN_Paral'
    NUM_CLASSES = 2
    EPOCHS = 300
    BATCH_SIZE = 32
    CLASS_WEIGHTS = None
    FN_LP = 300
    DETREND_LAMBDA = 50
    TEST_FILES = files[[6,30,31,32,33,34,35]]
    
    DWT_WAVELET = 'db7'
    DWT_LEVELS = 5
    KERNEL_SIZE  = 3
    DROPOUT_RATE = 0.2
    DWT_TRAINABLE = False
    MOVING_AVG_WINDOW = 50
    
        

In [10]:
# Generate CWT-CNN configuration
config = CNNPa_Config()
config.display()


Configurations:
BATCH_SIZE                     32
CHANNELS                       ['LEFT_TA', 'LEFT_TS', 'LEFT_BF', 'LEFT_RF', 'RIGHT_TA', 'RIGHT_TS', 'RIGHT_BF', 'RIGHT_RF']
CLASS_WEIGHTS                  None
DETREND_LAMBDA                 50
DROPOUT_RATE                   0.2
DROP_WITH_ZSCORE               None
DWT_LEVELS                     5
DWT_TRAINABLE                  False
DWT_WAVELET                    db7
EPOCHS                         300
FN_HP                          None
FN_IR                          False
FN_LP                          300
KERNEL_SIZE                    3
MOVING_AVG_WINDOW              50
NAME                           CNN_Paral
NUM_CLASSES                    2
N_ENV                          20
RECT                           False
REMOVE_FREQS                   True
SAME_LABEL                     True
SCALE                          True
SHUFFLE                        True
STEP_SIZE                      512
TEST_FILES                     ['G08_FoG_1_tr

## Data generate

In [6]:
data = Dataset(config)

In [7]:
# Load data from files
data.load_data(files)

X_train,Y_train,_ = data.train_set
X_valid,Y_valid,_ = data.valid_set
X_test, Y_test, _ = data.test_set

skip
skip
3/174: G06_FoG_trial_1_emg.csv
4/174: G06_FoG_trial_2_emg.csv
5/174: G06_FoG_trial_3_emg.csv
6/174: G07_Freezing_Trial1_trial_1_emg.csv
7/174: G08_FoG_1_trial_1_emg.csv
8/174: G08_FoG_2_trial_1_emg.csv
9/174: G11_FoG_trial_1_emg.csv
10/174: G11_FoG_trial_2_emg.csv
11/174: P379_M050_2_OFF_A_FoG_trial_1_emg.csv
12/174: P379_M050_2_OFF_A_FoG_trial_2_emg.csv
13/174: P379_M050_2_OFF_A_FoG_trial_3_emg.csv
14/174: P379_M050_2_OFF_B_FoG_trial_1_emg.csv
15/174: P379_M050_2_OFF_B_FoG_trial_2_emg.csv
16/174: P379_M050_2_OFF_B_FoG_trial_3_emg.csv
17/174: P551_M050_2_A_FoG_trial_1_emg.csv
18/174: P551_M050_2_B_FoG_trial_1_emg.csv
19/174: P551_M050_2_B_FoG_trial_2_emg.csv
20/174: P812_M050_2_B_FoG_trial_1_emg.csv
21/174: P812_M050_2_B_FoG_trial_2_emg.csv
22/174: normal/G02_Walking_trial_1_emg.csv
23/174: normal/G03_Walking_trial_1_emg.csv
24/174: normal/G03_Walking_trial_2_emg.csv
25/174: normal/G05_Walking_struct_fixed_trial_1_emg.csv
26/174: normal/G05_Walking_struct_fixed_trial_2_emg.cs

171/174: normal/P940_MSham_A_Walking_trial_6_emg.csv
172/174: normal/P940_MSham_B_Walking_trial_2_emg.csv
173/174: normal/P940_MSham_B_Walking_trial_4_emg.csv
174/174: normal/P940_MSham_B_Walking_trial_6_emg.csv


## Model

In [11]:
# Override base class of SimpleMode for CNNPa
class CNNPa_Model(NNModel):
        
    def make_wavelet_expansion(self, input_tensor):
        #input_tensor = Reshape((input_tensor.shape[-1],1))(input_tensor)
        print(input_tensor.shape)
        low_pass, high_pass  = pywt.Wavelet(self.wavelet_mother).filter_bank[:2]
        low_pass_filter = np.array(low_pass)
        high_pass_filter = np.array(high_pass)
        n_levels = self.wavelet_levels
        trainable=self.wavelet_trainable
        
        wv_kwargs = {
            "filters":1,
            "kernel_size":len(low_pass),
            "strides":2, 
            "use_bias":False, 
            "padding":"same", 
            "trainable":trainable,
        }

        approximation_coefficients = []
        detail_coefficients = []

        last_approximant = input_tensor
        last_approximant = K.reshape(last_approximant,(-1,last_approximant.shape[-2],last_approximant.shape[-1],1))
        for i in range(n_levels):
            lpf = low_pass_filter
            hpf = high_pass_filter
            #print(lpf.reshape((-1, int(self.channels))).shape)
            for j in range(self.channels):
                if j == 0:
                    a_n = KL.Conv1D(
                        kernel_initializer=keras.initializers.Constant(lpf.reshape((-1, 1))),
                        name="low_pass_{}.{}".format(i,j),
                        **wv_kwargs
                    )(last_approximant[:,:,j,:])
                    d_n = KL.Conv1D(
                        kernel_initializer=keras.initializers.Constant(hpf.reshape((-1, 1))),
                        name="high_pass_{}.{}".format(i,j),
                        **wv_kwargs,
                    )(last_approximant[:,:,j,:])
                else:
                #print(a_n)
                    temp_a = KL.Conv1D(
                        kernel_initializer=keras.initializers.Constant(lpf.reshape((-1, 1))),
                        name="low_pass_{}.{}".format(i,j),
                        **wv_kwargs
                    )(last_approximant[:,:,j,:])
                    a_n = K.concatenate([a_n,temp_a],axis=2)
                    temp_d = KL.Conv1D(
                        kernel_initializer=keras.initializers.Constant(hpf.reshape((-1, 1))),
                        name="high_pass_{}.{}".format(i,j),
                        **wv_kwargs,
                    )(last_approximant[:,:,j,:])
                    d_n = K.concatenate([d_n,temp_d],axis=2)
            detail_coefficients.append(d_n)
            approximation_coefficients.append(a_n)
            last_approximant = a_n
            last_approximant = K.reshape(last_approximant,(-1,last_approximant.shape[-2],last_approximant.shape[-1],1))

        return approximation_coefficients, detail_coefficients
    
    def envelopes(self, args):
        """Reparameterization trick by sampling fr an isotropic unit Gaussian.

        # Arguments
            args (tensor): mean and log of variance of Q(z|X)

        # Returns
            z (tensor): sampled latent vector
        """
        input_ = args
        abs_envelope = K.abs(input_)
        envelope = tf.signal.frame(
            abs_envelope,
            self.moving_avg_window,
            1,#steps
            pad_end=True,
            pad_value=0,
            axis=1,
            name='envelope_moving_average'
        )
        #print(envelope.shape)
        #envelope_reshaped = K.reshape(envelope,(-1,self.window_size,self.moving_avg_window))
        envelope_mean = K.mean(envelope, axis=2, keepdims=True)
        #print(envelope_mean.shape)
        envelope_mean = K.reshape(envelope_mean,(-1,self.window_size,self.channels))
        return envelope_mean
    
    def rfft_layer(self,data):
        #data=Reshape((data.shape[1],data.shape[2],1))(data)
        n = data.shape[2]
        #print(data.shape)
        for i in range(n):
            if i == 0:
                fft = K.abs(tf.signal.rfft(data[:,:,i]))
                fft = K.reshape(fft,(-1,fft.shape[1],1))
            else:
                temp = K.abs(tf.signal.rfft(data[:,:,i]))
                temp = K.reshape(temp,(-1,temp.shape[1],1))
                fft = K.concatenate([fft,temp],axis=2)
        #print(fft.shape)
        return fft
    
    def build(self,config):
        
        self.kernel_size = config.KERNEL_SIZE
        self.dropout_rate = config.DROPOUT_RATE  # Dropout rate
        # Define parameters for wavelet decomposition
        self.wavelet_mother = config.DWT_WAVELET 
        self.wavelet_levels = config.DWT_LEVELS  
        self.wavelet_trainable= config.DWT_TRAINABLE

        self.moving_avg_window = config.MOVING_AVG_WINDOW
        self.classes_num = config.NUM_CLASSES
        
        self.window_size = config.WINDOW_SIZE
        self.channels = len(config.CHANNELS)
        
        self.input_shape = (self.window_size,self.channels)
        
        input_ = KL.Input(shape=self.input_shape)
        
        #CNN on raw signal
        cnn_1 = KL.Conv1D(16, kernel_size=self.kernel_size, strides=1, padding="same", name='raw_conv_1')(input_)        
        cnn_1 = KL.ReLU()(cnn_1)
        #cnn_1 = KL.LeakyReLU(alpha=0.2)(cnn_1)
        cnn_1 = KL.MaxPooling1D(2)(cnn_1)
        cnn_1 = KL.Dropout(self.dropout_rate)(cnn_1)
        
        cnn_2 = KL.Conv1D(32, kernel_size=self.kernel_size, strides=1, padding="same", name='raw_conv_2')(cnn_1)
        cnn_2 = KL.BatchNormalization(momentum=0.8)(cnn_2)
        cnn_2 = KL.ReLU()(cnn_2)
        #cnn_2 = KL.LeakyReLU(alpha=0.2)(cnn_2)
        cnn_2 = KL.MaxPooling1D(2)(cnn_2)
        cnn_2 = KL.Dropout(self.dropout_rate)(cnn_2)
        #cnn_2 = KL.MaxPooling2D(2)(cnn_2)
        
        cnn_3 = KL.Conv1D(64, kernel_size=self.kernel_size, strides=1, padding="same", name='raw_conv_3')(cnn_2)
        cnn_3 = KL.BatchNormalization(momentum=0.8)(cnn_3)
        cnn_3 = KL.ReLU()(cnn_3)
        #cnn_3 = KL.LeakyReLU(alpha=0.2)(cnn_3)
        cnn_3 = KL.MaxPooling1D(2)(cnn_3)
        cnn_3 = KL.Dropout(self.dropout_rate)(cnn_3)
        #cnn_3 = KL.MaxPooling2D(2)(cnn_3)
        
        cnn_4_out = KL.Conv1D(32, kernel_size=self.kernel_size, strides=1, padding="same", name='raw_conv_4')(cnn_3)
        cnn_4 = KL.BatchNormalization(momentum=0.8)(cnn_4_out)
        cnn_4 = KL.ReLU()(cnn_4)
        #cnn_4 = KL.LeakyReLU(alpha=0.2)(cnn_4)
        cnn_4 = KL.MaxPooling1D(2)(cnn_4)
        cnn_4 = KL.Dropout(self.dropout_rate)(cnn_4)
        
        cnn_5 = KL.Conv1D(16, kernel_size=self.kernel_size, strides=1, padding="same", name='raw_conv_5')(cnn_4)
        cnn_5 = KL.BatchNormalization(momentum=0.8)(cnn_5)
        cnn_5 = KL.ReLU()(cnn_5)
        #cnn_5 = KL.LeakyReLU(alpha=0.2)(cnn_5)
        cnn_5 = KL.MaxPooling1D(2)(cnn_5)
        cnn_5 = KL.Dropout(self.dropout_rate)(cnn_5)
        #cnn_3 = KL.MaxPooling2D(2)(cnn_3)
        
#         cnn_5 = KL.Flatten()(cnn_5)
        cnn_5 = KL.GlobalAveragePooling1D()(cnn_5)
        
        #CNN on FFT of raw signal
        fft = KL.Lambda(self.rfft_layer,name='rfft')(input_)
        #fft = KL.Reshape((fft.shape[1]*fft.shape[2],1))(fft)
        #fft_abs = KL.Lambda(K.abs)(fft)
        #fft_abs = KL.Reshape((fft_abs.shape[-1],1), name='fft_abs')(fft_abs)
        #fft_abs = KL.Reshape((fft.shape[-2],fft.shape[-1],1), name='fft_abs')(fft)
        fft_cnn_1 = KL.Conv1D(16, kernel_size=self.kernel_size, strides=1, padding="same", name='fft_conv_1')(fft)
        fft_cnn_1 = KL.ReLU()(fft_cnn_1)
        #fft_cnn_1 = KL.LeakyReLU(alpha=0.2)(fft_cnn_1)
        fft_cnn_1 = KL.MaxPooling1D(2)(fft_cnn_1)
        fft_cnn_1 = KL.Dropout(self.dropout_rate)(fft_cnn_1)
        #fft_cnn_1 = KL.MaxPooling2D(2)(fft_cnn_1)
        
        fft_cnn_2 = KL.Conv1D(32, kernel_size=self.kernel_size, strides=1, padding="same", name='fft_conv_2')(fft_cnn_1)
        fft_cnn_2 = KL.BatchNormalization(momentum=0.8)(fft_cnn_2)
        fft_cnn_2 = KL.ReLU()(fft_cnn_2)
        #fft_cnn_2 = KL.LeakyReLU(alpha=0.2)(fft_cnn_2)
        fft_cnn_2 = KL.MaxPooling1D(2)(fft_cnn_2)
        fft_cnn_2 = KL.Dropout(self.dropout_rate)(fft_cnn_2)
        #fft_cnn_2 = KL.MaxPooling2D(2)(fft_cnn_2)
        
        fft_cnn_3 = KL.Conv1D(64, kernel_size=self.kernel_size, strides=1, padding="same", name='fft_conv_3')(fft_cnn_2)
        fft_cnn_3 = KL.BatchNormalization(momentum=0.8)(fft_cnn_3)
        fft_cnn_3 = KL.ReLU()(fft_cnn_3)
        #fft_cnn_3 = KL.LeakyReLU(alpha=0.2)(fft_cnn_3)
        fft_cnn_3 = KL.MaxPooling1D(2)(fft_cnn_3)
        fft_cnn_3 = KL.Dropout(self.dropout_rate)(fft_cnn_3)
        #fft_cnn_3 = KL.MaxPooling2D(2)(fft_cnn_3)
        
        fft_cnn_4_out = KL.Conv1D(32, kernel_size=self.kernel_size, strides=1, padding="same", name='fft_conv_4')(fft_cnn_3)
        fft_cnn_4 = KL.BatchNormalization(momentum=0.8)(fft_cnn_4_out)
        fft_cnn_4 = KL.ReLU()(fft_cnn_4)
        #fft_cnn_4 = KL.LeakyReLU(alpha=0.2)(fft_cnn_4)
        fft_cnn_4 = KL.MaxPooling1D(2)(fft_cnn_4)
        fft_cnn_4 = KL.Dropout(self.dropout_rate)(fft_cnn_4)   
        
        fft_cnn_5 = KL.Conv1D(16, kernel_size=self.kernel_size, strides=1, padding="same", name='fft_conv_5')(fft_cnn_4)
        fft_cnn_5 = KL.BatchNormalization(momentum=0.8)(fft_cnn_5)
        fft_cnn_5 = KL.ReLU()(fft_cnn_5)
        #fft_cnn_5 = KL.LeakyReLU(alpha=0.2)(fft_cnn_5)
        fft_cnn_5 = KL.MaxPooling1D(2)(fft_cnn_5)
        fft_cnn_5 = KL.Dropout(self.dropout_rate)(fft_cnn_5)
        #fft_cnn_3 = KL.MaxPooling2D(2)(fft_cnn_3)
        
#         fft_cnn_5 = KL.Flatten()(fft_cnn_5)
        fft_cnn_5 = KL.GlobalAveragePooling1D()(fft_cnn_5)
                
        #CNN on FFT of envelope
        envelope_window = KL.Lambda(self.envelopes, output_shape=(self.input_shape[0],self.input_shape[1]), name='envelope')(input_)
        #print(envelope_window.shape)
        #envelope_window = KL.Flatten()(envelope_window)
        envelope_fft = KL.Lambda(self.rfft_layer,name='envelope_fft')(envelope_window)
        #envelope_fft_abs = KL.Lambda(K.abs)(envelope_fft)
        #envelope_fft_abs = KL.Reshape((envelope_fft_abs.shape[-1],1))(envelope_fft_abs)
        #envelope_fft_abs = KL.Reshape((envelope_fft.shape[-2],envelope_fft.shape[-1],1))(envelope_fft)
        #envelope_fft =KL.Reshape((envelope_fft.shape[1]*envelope_fft.shape[2],1))(envelope_fft)
        
        envelope_cnn_1 = KL.Conv1D(16, kernel_size=self.kernel_size, strides=1, padding="same", name='fft_env_conv_1')(envelope_fft)
        envelope_cnn_1 = KL.ReLU()(envelope_cnn_1)
        #envelope_cnn_1 = KL.LeakyReLU(alpha=0.2)(envelope_cnn_1)
        envelope_cnn_1 = KL.MaxPooling1D(2)(envelope_cnn_1)
        envelope_cnn_1 = KL.Dropout(self.dropout_rate)(envelope_cnn_1)
        #envelope_cnn_1 = KL.MaxPooling2D(2)(envelope_cnn_1)
        
        envelope_cnn_2 = KL.Conv1D(32, kernel_size=self.kernel_size, strides=1, padding="same", name='fft_env_conv_2')(envelope_cnn_1)
        envelope_cnn_2 = KL.BatchNormalization(momentum=0.8)(envelope_cnn_2)
        envelope_cnn_2 = KL.ReLU()(envelope_cnn_2)
        #envelope_cnn_2 = KL.LeakyReLU(alpha=0.2)(envelope_cnn_2)
        envelope_cnn_2 = KL.MaxPooling1D(2)(envelope_cnn_2)
        envelope_cnn_2 = KL.Dropout(self.dropout_rate)(envelope_cnn_2)
        #envelope_cnn_2 = KL.MaxPooling2D(2)(envelope_cnn_2)
        
        envelope_cnn_3 = KL.Conv1D(64, kernel_size=self.kernel_size, strides=1, padding="same", name='fft_env_conv_3')(envelope_cnn_2)
        envelope_cnn_3 = KL.BatchNormalization(momentum=0.8)(envelope_cnn_3)
        envelope_cnn_3 = KL.ReLU()(envelope_cnn_3)
        #envelope_cnn_3 = KL.LeakyReLU(alpha=0.2)(envelope_cnn_3)
        envelope_cnn_3 = KL.MaxPooling1D(2)(envelope_cnn_3)
        envelope_cnn_3 = KL.Dropout(self.dropout_rate)(envelope_cnn_3)
        #envelope_cnn_3 = KL.MaxPooling2D(2)(envelope_cnn_3)
        
        envelope_cnn_4_out = KL.Conv1D(32, kernel_size=self.kernel_size, strides=1, padding="same", name='fft_env_conv_4')(envelope_cnn_3)
        envelope_cnn_4 = KL.BatchNormalization(momentum=0.8)(envelope_cnn_4_out)
        envelope_cnn_4 = KL.ReLU()(envelope_cnn_4)
        #envelope_cnn_4 = KL.LeakyReLU(alpha=0.2)(envelope_cnn_4)
        envelope_cnn_4 = KL.MaxPooling1D(2)(envelope_cnn_4)
        envelope_cnn_4 = KL.Dropout(self.dropout_rate)(envelope_cnn_4)
        
        envelope_cnn_5 = KL.Conv1D(16, kernel_size=self.kernel_size, strides=1, padding="same", name='fft_env_conv_5')(envelope_cnn_4)
        envelope_cnn_5 = KL.BatchNormalization(momentum=0.8)(envelope_cnn_5)
        envelope_cnn_5 = KL.ReLU()(envelope_cnn_5)
        #envelope_cnn_5 = LeakyReLU(alpha=0.2)(envelope_cnn_5)
        envelope_cnn_5 = KL.MaxPooling1D(2)(envelope_cnn_5)
        envelope_cnn_5 = KL.Dropout(self.dropout_rate)(envelope_cnn_5)
        #envelope_cnn_3 = KL.MaxPooling2D(2)(envelope_cnn_3)
        
#         envelope_cnn_5 = KL.Flatten()(envelope_cnn_5)
        envelope_cnn_5 = KL.GlobalAveragePooling1D()(envelope_cnn_5)
        
        # Wavelet Expansion
        approx_stack, detail_stack = self.make_wavelet_expansion(input_)
        features_list = []
        features_list.extend(detail_stack)
        features_list.append(approx_stack[-1])
        wavelet_concatenate = KL.Concatenate(axis=1,name='wavelet_concat')(features_list)
        wavelet_concatenate = K.abs(wavelet_concatenate)
        wavelet_cnn_1 = KL.Conv1D(16, kernel_size=self.kernel_size, strides=1, padding="same", name='wavelet_conv_1')(wavelet_concatenate)
        wavelet_cnn_1 = KL.ReLU()(wavelet_cnn_1)
        #wavelet_cnn_1 = KL.LeakyReLU(alpha=0.2)(wavelet_cnn_1)
        wavelet_cnn_1 = KL.MaxPooling1D(2)(wavelet_cnn_1)
        wavelet_cnn_1 = KL.Dropout(self.dropout_rate)(wavelet_cnn_1)
        
        wavelet_cnn_2 = KL.Conv1D(32, kernel_size=self.kernel_size, strides=1, padding="same", name='wavelet_conv_2')(wavelet_cnn_1)
        wavelet_cnn_2 = KL.BatchNormalization(momentum=0.8)(wavelet_cnn_2)
        wavelet_cnn_2 = KL.ReLU()(wavelet_cnn_2)
        #wavelet_cnn_2 = KL.LeakyReLU(alpha=0.2)(wavelet_cnn_2)
        wavelet_cnn_2 = KL.MaxPooling1D(2)(wavelet_cnn_2)
        wavelet_cnn_2 = KL.Dropout(self.dropout_rate)(wavelet_cnn_2)
        
        wavelet_cnn_3 = KL.Conv1D(64, kernel_size=self.kernel_size, strides=1, padding="same", name='wavelet_conv_3')(wavelet_cnn_2)
        wavelet_cnn_3 = KL.BatchNormalization(momentum=0.8)(wavelet_cnn_3)
        wavelet_cnn_3 = KL.ReLU()(wavelet_cnn_3)
        #wavelet_cnn_3 = KL.LeakyReLU(alpha=0.2)(wavelet_cnn_3)
        wavelet_cnn_3 = KL.MaxPooling1D(2)(wavelet_cnn_3)
        wavelet_cnn_3 = KL.Dropout(self.dropout_rate)(wavelet_cnn_3)
        
        wavelet_cnn_4_out = KL.Conv1D(32, kernel_size=self.kernel_size, strides=1, padding="same", name='wavelet_conv_4')(wavelet_cnn_3)
        wavelet_cnn_4 = KL.BatchNormalization(momentum=0.8)(wavelet_cnn_4_out)
        wavelet_cnn_4 = KL.ReLU()(wavelet_cnn_4)
        #wavelet_cnn_4 = KL.LeakyReLU(alpha=0.2)(wavelet_cnn_4)
        wavelet_cnn_4 = KL.MaxPooling1D(2)(wavelet_cnn_4)
        wavelet_cnn_4 = KL.Dropout(self.dropout_rate)(wavelet_cnn_4)
        
        wavelet_cnn_5 = KL.Conv1D(16, kernel_size=self.kernel_size, strides=1, padding="same", name='wavelet_conv_5')(wavelet_cnn_4)
        wavelet_cnn_5 = KL.BatchNormalization(momentum=0.8)(wavelet_cnn_5)
        wavelet_cnn_5 = KL.ReLU()(wavelet_cnn_5)
        #wavelet_cnn_5 = KL.LeakyReLU(alpha=0.2)(wavelet_cnn_5)
        wavelet_cnn_5 = KL.MaxPooling1D(2)(wavelet_cnn_5)
        wavelet_cnn_5 = KL.Dropout(self.dropout_rate)(wavelet_cnn_5)
        
#         wavelet_cnn_5 = KL.Flatten()(wavelet_cnn_5)
        wavelet_cnn_5 = KL.GlobalAveragePooling1D()(wavelet_cnn_5)
        
        concatenate = KL.Concatenate()([cnn_5,envelope_cnn_5,fft_cnn_5,wavelet_cnn_5])

        dropout = KL.Dropout(self.dropout_rate)(concatenate)
        mlp = KL.Dense(self.classes_num,activation='softmax')(dropout)
                
        model = KM.Model(input_, mlp)
        
        model.summary()
        
        if config.COST_SENSITIVE:
            self.cost_matrix = config.COST_MATRIX
            model.compile(loss=self.sparse_cost_sensitive_loss, optimizer="adam", metrics=['accuracy'])
            print('Using cost sensitive with cost matrix:\n',np.array(self.cost_matrix))
        else:
            model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
            if config.CLASS_WEIGHTS != None:
                print('Using categorical crossentropy with class weights:\n',config.CLASS_WEIGHTS)
            else:
                print('Using categorical crossentropy without class weights.')
        
        return model

    def sparse_cost_sensitive_loss (self,y_true,y_pred):
        cost_matrix = self.cost_matrix
        batch_cost_matrix = tf.nn.embedding_lookup(cost_matrix, tf.argmax(y_true,axis=1))
        eps = 1e-6
        probability = tf.clip_by_value(y_pred, eps, 1-eps)
        cost_values = tf.math.log(1-probability)*batch_cost_matrix
        loss = tf.reduce_mean(-tf.reduce_sum(cost_values, axis=1))
        return loss

## Data split

In [13]:
# data split and processing for model
class_id = [2,6]
binary = True
x_train,y_train,x_valid,y_valid,x_test,y_test = utils.data_split_oh((X_train,X_valid,X_test),
                                   (Y_train,Y_valid,Y_test),
                                   class_id,
                                   binary,
                                   random_state = 555)

NameError: name 'X_train' is not defined

## Model training

In [14]:
#cost_matrix = tf.constant([[0,1.5,1,1.2],
#              [1,0,1,1],
#              [5,10,0,5],
#              [1.,1.,1,0]])
#cost_matrix = tf.constant([[0.,1.,1.],
#              [10.,0.,1.],
#              [10.,4.,0.]])
config.COST_MATRIX = tf.constant([[0,1.],
              [10,0]])

config.DWT_WAVELET = 'db7'
config.DWT_LEVELS = 3
config.KERNEL_SIZE  = 3
config.DROPOUT_RATE = 0.2
config.DWT_TRAINABLE = False
config.MOVING_AVG_WINDOW = 20

if binary:
    config.COST_SENSITIVE = True
    config.NUM_CLASSES = 2
else:
    config.COST_SENSITIVE = False
    config.NUM_CLASSES = len(class_id)

# Generate CWT_CNN Model
cnnpa_model = CNNPa_Model('CNNPa',config,'./model/CNNPa/')

(None, 1024, 8)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1024, 8)]    0                                            
__________________________________________________________________________________________________
tf.reshape (TFOpLambda)         (None, 1024, 8, 1)   0           input_1[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 1024, 1)      0           tf.reshape[0][0]                 
__________________________________________________________________________________________________
tf.__operators__.getitem_2 (Sli (None, 1024, 1)      0           tf.reshape[0][0]                 
______________________________________________________________________________

In [11]:
early_stopping = keras.callbacks.EarlyStopping(patience = 10,
                                             monitor = 'val_loss', 
                                             #baseline = 0.9,
                                             restore_best_weights=True)
cnnpa_model.train((x_train,y_train),(x_valid,y_valid),config.EPOCHS,config.BATCH_SIZE,[early_stopping])


Starting at epoch 0.

Checkpoint Path: ./model/CNNPa/cnn_paral20210120T0019\CNNPa_cnn_paral_{epoch:04d}.h5
Epoch 1/300
14/14 [==============================] - 25s 845ms/step - loss: 0.6663 - accuracy: 0.5907 - val_loss: 0.5078 - val_accuracy: 0.8227
Epoch 2/300
14/14 [==============================] - 4s 284ms/step - loss: 0.5476 - accuracy: 0.6813 - val_loss: 0.4224 - val_accuracy: 0.8723
Epoch 3/300
14/14 [==============================] - 4s 315ms/step - loss: 0.5210 - accuracy: 0.7263 - val_loss: 0.3721 - val_accuracy: 0.8723
Epoch 4/300
14/14 [==============================] - 4s 261ms/step - loss: 0.4275 - accuracy: 0.7935 - val_loss: 0.3260 - val_accuracy: 0.9078
Epoch 5/300
14/14 [==============================] - 4s 297ms/step - loss: 0.3853 - accuracy: 0.8270 - val_loss: 0.3014 - val_accuracy: 0.9220
Epoch 6/300
14/14 [==============================] - 4s 260ms/step - loss: 0.3434 - accuracy: 0.8707 - val_loss: 0.2549 - val_accuracy: 0.9291
Epoch 7/300
14/14 [==============

## Model evaluation

In [12]:
acc_train,cm_train = cnnpa_model.model_metrics(x_train,y_train)
acc_valid,cm_valid = cnnpa_model.model_metrics(x_valid,y_valid)
acc_test,cm_test = cnnpa_model.model_metrics(x_test,y_test)
print('acc_train: %f\nconfusion_matrix:\n'%acc_train,cm_train)
print('acc_valid: %f\nconfusion_matrix:\n'%acc_valid,cm_valid)
print('acc_test: %f\nconfusion_matrix:\n'%acc_test,cm_test)

acc_train: 0.971429
confusion_matrix:
 [[256   3]
 [  9 152]]
acc_valid: 0.950355
confusion_matrix:
 [[93  3]
 [ 4 41]]
acc_test: 0.939597
confusion_matrix:
 [[57  0]
 [ 9 83]]
